## Introducción al Aprendizaje por Refuerzo en entornos discretos
##### Belén Díaz Agudo 

### Ejemplo 2: Frozen Lake
En el entorno FrozenLake-v0 https://gym.openai.com/envs/FrozenLake-v0/ el agente controla el movimiento de un personaje en un mundo de rejilla. Algunas baldosas son transitables (walkable) y otras hacen que el agente caiga al agua. 
__La dirección de movimiento del agente es incierta y solo depende parcialmente de la dirección elegida (porque puede resbalar en el hielo)__. 
La recompensa se obtiene cuando el agente llega a traves de un camino transitable a una casilla objetivo.

In [7]:
# Creamos el entorno FrozenLake-v1 y realizamos acciones aleatorias para resolver el problema. 
# Se muestra el número de pasos del episodio. 
# Un episodio puede terminar (done) con éxito si ha llegado a la casilla final, o con fallo si ha caido al agua.
# Se ejecutan 20 episodios. Observa si termina con éxito en alguno de ellos. 

environment_name = "FrozenLake-v1"
env = gym.make(environment_name).env
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)
        action = env.action_space.sample()
        observation, reward, done, info  = env.step(action)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
Episode finished after 16 timesteps

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Episode finished after 7 timesteps

SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
Episode finished after 6 timesteps

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF


In [8]:
# Observamos el tamaño del espacio de estados
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(4)
State Space Discrete(16)


### Ejercicio: configura el aprendizaje por refuerzo para resolver el problema de Frozen Lake.
#### Utiliza distintos valores de configuración de los parámetros y observa qué configuración se comporta mejor. ¿qué métrica has utilizado para determinar si una configuración es buena? 
#### Observa cuál es la función de recompensa que se define por defecto (consulta en la documentación de gym).  Mejórala reescribiendo el valor de reward y observa cómo afecta la mejora de la función de recompensa en el proceso de aprendizaje.  Explica cómo has medido esta mejora.


#### Puedes entregar tus resultados de forma opcional: 

- Métrica utilizada: 
- Función de recompensa mejorada: 
- Mejor configuración de parámetros 
- Resumen de resultados (los mejores obtenidos) con la métrica anterior: 